# Exercice 2 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

# Example sentence embeddings (4 dimensions per word)
embeddings = {
    "I": [1, 0, 0, 0],
    "love": [0, 1, 0, 0],
    "playing": [0, 0, 1, 0],
    "tennis": [0, 0, 0, 1],
    "football": [1, 1, 0, 0],
    "often": [1, 0, 1, 0],
    "with": [0, 1, 1, 0],
    "friends": [1, 1, 1, 0],
    "and": [0, 0, 1, 1]
}


# Convert sentence to sequence of embeddings
phrases = [
    ["I", "love", "playing", "football", "often", "with", "friends"],
    ["I", "love", "tennis"]
]


max_len = max(len(s) for s in phrases)

liste_inputs = []
for ligne in phrases:
    emb = [embeddings[mot] for mot in ligne]
    #padding selon longueur phrase
    while len(emb) < max_len:
        emb.append([0, 0, 0, 0])
    liste_inputs.append(np.array(emb))
    

inputs = np.array(liste_inputs)



# Initialize weights for Query, Key, and Value matrices
np.random.seed(0)
Wq = np.random.rand(4, 4)
Wk = np.random.rand(4, 4)
Wv = np.random.rand(4, 4)

for i, phrase_inputs in enumerate(inputs): 
    print("\n\n------------------------------")
    print(f"\nANALYSE DE LA PHRASE {i+1}: {phrases[i]}")
    
    vrai_longueur = len(phrases[i])
    
    # Compute Query, Key, and Value matrices
    Q = np.dot(phrase_inputs, Wq)
    K = np.dot(phrase_inputs, Wk)
    V = np.dot(phrase_inputs, Wv)

    # Compute attention scores
    scores = np.dot(Q, K.T) / np.sqrt(K.shape[1])



     #masquage
    scores_masquer = scores.copy()
    scores_masquer[:, vrai_longueur:] = -1000000000000
    scores_masquer[vrai_longueur:, :] = -1000000000000  
    
    attention_weights_sans_mask = softmax(scores)
    attention_weights_mask = softmax(scores_masquer)
    
    #pour eviter NAN dans les calcul et les plot ici on force a 0
    attention_weights_mask[vrai_longueur:, :] = 0.0
      
    output_sans_mask = np.dot(attention_weights_sans_mask, V)
    output_mask = np.dot(attention_weights_mask, V)
    
    
    phrase_label = phrases[i] + ['padding'] * (max_len - vrai_longueur)


    # Plotting the inputs
    plt.figure(figsize=(20, 10))
    plt.suptitle(f"Phrase: {' '.join(phrases[i])}", fontsize=16)
    
    plt.subplot(2, 4, 1)
    sns.heatmap(phrase_inputs, annot=True, cmap='viridis', xticklabels=phrase_label, yticklabels=phrase_label)
    plt.title('Inputs')

    # Plotting Query (Q) matrix
    plt.subplot(2, 4, 2)
    sns.heatmap(Q, annot=True, cmap='viridis', xticklabels=['dim1', 'dim2', 'dim3', 'dim4'], yticklabels=phrase_label)
    plt.title('Query (Q)')

    # Plotting Key (K) matrix
    plt.subplot(2, 4, 3)
    sns.heatmap(K, annot=True, cmap='viridis', xticklabels=['dim1', 'dim2', 'dim3', 'dim4'], yticklabels=phrase_label)
    plt.title('Key (K)')

    # Plotting Value (V) matrix
    plt.subplot(2, 4, 4)
    sns.heatmap(V, annot=True, cmap='viridis', xticklabels=['dim1', 'dim2', 'dim3', 'dim4'], yticklabels=phrase_label)
    plt.title('Value (V)')

    # Plotting Attention Scores  Sans masque 
    plt.subplot(2, 4, 5)
    sns.heatmap(scores, annot=True, cmap='viridis', xticklabels=phrase_label, yticklabels=phrase_label)
    plt.title('Attention Scores Sans masque')

    # Plotting Attention Weights Sans masque 
    plt.subplot(2, 4, 6)
    sns.heatmap(attention_weights_sans_mask, annot=True, cmap='viridis', xticklabels=phrase_label, yticklabels=phrase_label)
    plt.title('Attention Weights Sans masque')



    #Attention Weights AVEC masque
    plt.subplot(2, 4, 7)
    sns.heatmap(attention_weights_mask, annot=True, cmap='viridis', xticklabels=phrase_label, yticklabels=phrase_label)
    plt.title('Attention Scores Sans masque')

    #Output AVEC masque
    plt.subplot(2, 4, 8)
    sns.heatmap(output_mask, annot=True, cmap='viridis', xticklabels=['dim1', 'dim2', 'dim3', 'dim4'], yticklabels=phrase_label)
    plt.title('Output AVEC masque')



    plt.tight_layout()
    plt.show()

    print("Phrase:", phrases[i])
    print("Vraie longueur: ",vrai_longueur, " mots")
    print("\nInputs:\n", phrase_inputs)
    print("\nQuery (Q):\n", Q)
    print("\nKey (K):\n", K)
    print("\nValue (V):\n", V)
    print("\nAttention Scores SANS masque:\n", scores)
    print("\nAttention Weights SANS masque:\n", attention_weights_sans_mask)

    print("\nAttention Weights AVEC masque:\n", attention_weights_mask)
    print("\nOutput AVEC masque:\n", output_mask)
    
    
    
    



print("\n\n\n-------------------------")
print("Comparaison Embedding: random et pretrained")

#embeddings aléatoires
np.random.seed(42)
embeddings_aleatoires = {word: np.random.randn(4) for word in embeddings.keys()}

test_phrase = ["I", "love", "tennis"]

#avec embedding choisi
inputs_perso = np.array([embeddings[mot] for mot in test_phrase])
Q_perso = np.dot(inputs_perso, Wq)
scores_perso = np.dot(Q_perso, Q_perso.T) / np.sqrt(Q_perso.shape[1])

#avec embeddings aleatoires  
inputs_aleatoire = np.array([embeddings_aleatoires[mot] for mot in test_phrase])
Q_aleatoire = np.dot(inputs_aleatoire, Wq)
scores_aleatoire = np.dot(Q_aleatoire, Q_aleatoire.T) / np.sqrt(Q_aleatoire.shape[1])

print("Scores avec embeddings pretrained:\n", scores_perso)
print("\nScores avec embeddings ALEATOIRES:\n", scores_aleatoire)
print("\nLes embeddings aleatoires creent des patterns d'attention differents!")


plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.heatmap(scores_perso, annot=True, cmap="viridis")
plt.title("Attention  Pretrained Embeddings")

plt.subplot(1,2,2)
sns.heatmap(scores_aleatoire, annot=True, cmap="viridis")
plt.title("Attention  Random Embeddings")
plt.tight_layout()
plt.show()

**REPONSE EXERCICE 2 :**

**1.** La phrase plus longue "I love playing football often with friends" a été convertie en embeddings 4 bits personnalisé chaque mot est un vecteur comme : "I"=[1,0,0,0], "love"=[0,1,0,0]. Les matrices Query, Key et Value sont calculées grace à cela donc en multipliant ces embeddings par des poids Wq, Wv qui sont  aléatoire on voit que les embeddings deviennent utilisables pour l'attention

**2.** Sans asque, l'attention fait sur tous les tokens et aussi sur le padding gaspille de l'information. Avec masque, l'attention se fait que sur les mots réels donc pour "I love tennis" paddé que les 3 premières positions ont des poids non nuls, les 4 positions de padding sont à0.

**3.** En comparant mes embeddings structurés avec des embeddings aléatoire on voit une différence qui est que les embeddings structurés produisent des patterns d'attention logique alors que les embeddings aléatoires génèrent des scores qu'on peut pas prédire. C'est pour cela qu'il faut bien les initialiser pour comprendre une phrasee
